In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim

import pandas as pd
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force

import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation

import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from edward_tools import visualization
from IPython.display import HTML, Image
from test_NAND_platform_helper_function import *

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# import data

In [ ]:
experiment_idenitfication = f"Experiment 2 (2025-07-01)"
experiment_idenitfication = f"Experiment 1 (2025-07-15)"

experiment_idenitfication = f"Experiment 2 (2025-09-02)"
experiment_idenitfication = f"Experiment 2 (1990-09-09)"
experiment_idenitfication = f"Experiment 1 (2025-09-11)"
df = loadDataFrame()
# target = getDataByID(df, "4a95ff94e9632672bb8f2829d4d4d8a3c71804c7a2c076753e3a7d9d0535df67")
target = getDataByComment(df, experiment_idenitfication)
# target = getDataByID(df, "4a95ff94e9632672bb8f2829d4d4d8a3c71804c7a2c076753e3a7d9d0535df67")


In [ ]:
len(target)

In [ ]:
failure_simulation_id = target[target['number_of_failure_particles'] > 0]['simulation_id'].values


In [ ]:
sim_id_array = target['simulation_id'].values
failure_particle_array = []
for sim_id in failure_simulation_id:
    print(sim_id)
    if os.path.exists(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_failure_path.npy"):
        failure_particle_array.append(np.load(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_failure_path.npy"))

In [ ]:
failure_particle_array

In [ ]:
protocol_list = target.tail(1)['protocol_list_item'].values[0]
initial_parameter_dict = target.tail(1)['initial_parameter_dict'].values[0]
params = target.tail(1)['params'].values[0]
params['sim_params'] = np.array(params['sim_params'])
params['mass'] = np.array(params['mass'])


failure_particle_array = np.vstack(failure_particle_array)
init_state = failure_particle_array[:, 0, ...]



In [ ]:

# failure_path_file_path = f"../coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{experiment_comment.replace("/", "-")}_failure_path.npy"
# failure_particles = simResult['cfqr'].get_failure_particles(mapping_dict)

# if not os.path.isfile(failure_path_file_path):
#     np.save(failure_path_file_path, np.array([]))


# failure_particles_array = np.load(failure_path_file_path)


# if len(failure_particles) > 0 and len(failure_particles_array) == 0:
#     np.save(failure_path_file_path, failure_particles)

# if len(failure_particles) > 0 and len(failure_particles_array) > 0:
#     failure_particles_new_array = np.vstack([failure_particles_array, failure_particles])
#     np.save(failure_path_file_path, failure_particles_new_array)


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, storage_protocol= storage_protocol,  computation_protocol= comp_protocol,  protocol_list = protocol_list, has_velocity=True)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state)
print(f"mean KE = {np.mean(cfqr.system.get_kinetic_energy(init_state))}")

# Show animations

In [ ]:
contourData = {
    "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
    "contour_range": [600, 2000], "domain_limit": [-3, 3],
    "title": None
}

cutlineInformation = {
    "cutlineList": [(-1.3, "h","red"), (1.3, "h","blue")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [[600, 1200], [750, 980]]
}

particleInformation = {
    "showParticles": True,
    "project_axis": "phi_1",
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}

graph_setting = {
        0.1: {"vmin": 1000, "vmax": 2000, "ymin": 1000, "ymax": 3000},
        0.5: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        1: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        2: {"vmin": 50, "vmax": 250, "ymin": 50, "ymax": 250},    
        4.2: {"vmin": 600, "vmax": 2000, "ymin": 600, "ymax": 3000},
        5: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        7: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        20: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        100: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        1000: {"vmin": 0, "vmax": 1, "ymin": 0, "ymax": 1},
    }

pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}

In [ ]:
time_index_start = 0
time_index_end = -1
frame_skip = 500
frame_skip = 50
all_state = failure_particle_array[:, time_index_start:time_index_end, ...]
reducedTimeArray = cfqr.protocol_all_time_array[slice(*params["as_step"])]
targetTimeArray = reducedTimeArray[time_index_start:time_index_end]

animation_play = True
# animation_play = False
if animation_play:    
    animation_time_array = targetTimeArray[::frame_skip]
    animation_states = all_state[:, ::frame_skip, ...]
    print("geenerated at ", datetime.datetime.now())

    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(cfqr, time_array=animation_time_array, state_array = animation_states, plot_axis= [0,1], vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH", params=params, 
cutlineGeneratingFunction = False, cutlineInformation = cutlineInformation, particleInformation = particleInformation, contourData = contourData, pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    plt.close()
    


In [ ]:
Image(filename="scatter.gif")

In [ ]:
plt.plot(all_state[0, ..., 2, 0])
plt.hlines(2.04, xmin = 0, xmax = 25000)


# Show animations in different spaces

In [ ]:

# %%capture

# import random

# # initial_state_array = [f"saved_init_state/init_state_database/T=4.2K, N = 1000, beta = 1.35"]

# initial_state_array = [f"saved_init_state/init_state_database_2/T_4.2K_N_1000_beta_1.35"]
# initial_state_folder = initial_state_array[0]
# initial_state_folder_path = os.listdir(initial_state_folder)
# initial_state_KE = []
# # initial_state_chosen = os.path.join(initial_state_folder, random.choice(initial_state_folder_path))
# for init_state_path in initial_state_folder_path:
#     initial_state_chosen = os.path.join(initial_state_folder, init_state_path)
#     _init_state = np.load(initial_state_chosen)

#     cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, storage_protocol= storage_protocol,  computation_protocol= comp_protocol,  protocol_list = protocol_list, has_velocity=True)

#     protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
#     cfqr.initialize_sim()

#     cfqr.set_sim_attributes(init_state=_init_state)
#     initial_state_KE.append(np.mean(cfqr.system.get_kinetic_energy(_init_state)))


In [ ]:
# print(len(initial_state_folder_path))
# plt.hist(initial_state_KE, bins = 10)

In [ ]:
all_states = failure_particle_array
frame_skip = 10
dt = 1/1000
targetTimeArray = np.arange(failure_particle_array.shape[1]) * dt * 10
animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_states[:, ::frame_skip, ...]

fig, ax = plt.subplots(2, 5, figsize=[20, 7])
plot_label_1 = [("phi_1", "phi_2"), ("phi_1dc", "phi_2dc"), ("phi_1", "phi_1dc"), ("phi_1", "phi_2dc"), ("phi_2", "phi_1dc"), ("phi_2", "phi_2dc")]
plot_label_2 = ["phi_1", "phi_2", "phi_1dc", "phi_2dc"]
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}

particle_index = visualization.separate_by_state_2(initial_state[:, (0, 1), :])
    
ax_flatten = ax.flatten()
ax_flatten_1 = [ax_flatten[i] for i in [0, 1, 2, 5, 6, 7]]
ax_flatten_2 = [ax_flatten[i] for i in [3, 4, 8, 9]]
phi_1_array = np.linspace(-4, 4, 1000)
phi_2_array = np.linspace(-4, 4, 1000)
phi_1dc_array = np.linspace(-4, 4, 1000)
phi_2dc_array = np.linspace(-4, 4, 1000)

for _ax, (xlabel, ylabel) in zip(ax_flatten_1, plot_label_1):
    _ax.set_xlim(-4, 4)
    _ax.set_ylim(-4, 4)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel(ylabel)    
    _ax.set_aspect(1)

for _ax, xlabel in zip(ax_flatten_2, plot_label_2):
    _ax.set_xlim(-4, 4)
    _ax.set_ylim(500, 1200)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel("PE")    
    # _ax.set_aspect(1)


plot_axis_1 = [[0, 1], [2, 3], [0, 2], [0, 3], [1, 2], [1, 3]]
plot_axis_2 = [0, 1, 2, 3]

scatter_array_1 = []
scatter_array_2 = []
potential_min_plot_array = {}
potential_max_plot_array = {}

potential_min_2_plot_array = {}
potential_max_2_plot_array = {}

potential_min_1dc_plot_array = {}
potential_max_1dc_plot_array = {}

for key, index in particle_index.items():
# for key, index in [("01", particle_index["11"])]:
    for _i, (_x, _y) in enumerate(plot_axis_1):
        _scatter = ax_flatten_1[_i].scatter(all_states[index, 0, _x, 0], all_states[index, 0, _y, 0], c = pColor[key])
        scatter_array_1.append(_scatter)
        
def animate(i):
    counter_1, counter_2 = 0, 0
    _t = animation_time_array[i]

    for key, index in particle_index.items():
        for _i, (_x, _y) in enumerate(plot_axis_1):
            scatter_array_1[counter_1].set_offsets(np.c_[animation_states[index, i, _x, 0], animation_states[index, i, _y, 0]])
            counter_1 += 1

ani = FuncAnimation(fig, animate, interval=100, frames=len(animation_time_array))
writer = animation.PillowWriter(fps=30, metadata=dict(artist='Me'), bitrate=1800)
ani.save('failure_scatter.gif', writer=writer)
plt.close()

In [ ]:
particle_index

In [ ]:
Image(filename="failure_scatter.gif")

# parameters

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 100
L_factor = 0.35
# L_factor = 1
# I_m_factor = 50
I_m_factor = 1
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
R_1, R_2 = 371, 371                                # ohm
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 10
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "Experiment 3 (2024/2/29): test for offset energy"

protocol_key = ['U0_1', 'U0_2', 'gamma_1', 'gamma_2', 'beta_1', 'beta_2', 'd_beta_1', 'd_beta_2', 
                'phi_1_x', 'phi_2_x', 'phi_1_dcx', 'phi_2_dcx', 'M_12', 'x_c']


# define potential

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
phi_1_dcx , phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_x_off, phi_2_x_off = 0, 0
phi_1_dcx_off, phi_2_dcx_off, M_12_off = 0, 0, 0

if I_m_factor == 0:
    phi_1_x_on = 0
    phi_2_x_on = 0
else:
    phi_1_x_on = -0.008
    phi_2_x_on = -0.008
    # phi_1_x_on = -3
    # phi_2_x_on = -3

phi_1_x_on = 0
phi_2_x_on = 0
    
phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_on,  "phi_2_x": phi_2_x_on,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
U0_1/k_BT

In [ ]:
failure_particle_array